# Day 04

Start with learning about NumPy today.


https://aaltoscicomp.github.io/python-for-scicomp/numpy/

In [1]:
import numpy as np

In [2]:
a = list(range(10000))
b = [ 0 ] * 10000

In [3]:
%%timeit
for i in range(len(a)):
    b[i] = a[i]**2

2.62 ms ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
a = np.arange(10000)
b = np.zeros(10000)

In [5]:
%%timeit
b = a ** 2

3.53 µs ± 17.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Problem 1

Get the score of a winning board of bingo.
The score of a winning board can be calculated by multiplication of; 
a) the sum of values on the board that have not been called
b) the value of the last call

### Steps to take
1. read the text file -> it seems `loadtxt` function can be used. (ref. https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html)
2. the first line of the text file is to be stored as a list of items "calls" -> it seems `(array).astype('int')` can be used to transform dataformat from str to int.
3. store the values of each board as an array of 5x5
4. repeat 3 until the end of the file
5. make the same number of 5x5 array of all FALSE -> it seems `np.ones(5,5), bool)` can be used?
6. start with the first number of the "calls", find the first number in each array made in the step 3, **if** there is that number, **then** 1) make the corresponding value in the array made in step 5 into TRUE, and 2) stop finding process for this board.
7. check how many FALSE is left for each column and row. **If** either is 0, **then** stop all the finding process for all the boards. **Else**, continue to the next board and do the same thing of step 6 and 7, repeat to the next number in the "calls". 
8. when the process ends with the TRUE condition in the step 7, then get the sum of all the values at the FALSE position in the last board of the process.
9. multiply the sum obtained in step 8 and number used for the last call.


In [135]:
### Step 1 and 2
# get the "calls" list
calls = np.loadtxt("example.txt", dtype = 'int', delimiter = ',', max_rows = 1)
print(calls)
type(calls[0])

[ 7  4  9  5 11 17 23  2  0 14 21 24 10 16 13  6 15 25 12 22 18 20  8 19
  3 26  1]


numpy.int64

In [136]:
### Step 3
# how many boards are there?
boards_pre = np.loadtxt("example.txt", dtype = 'int', skiprows = 2)
total_boards = int(len(boards_pre)/5) 
print(total_boards)

3


In [137]:
### Step 4
# prepare bords to store
boards = np.zeros((total_boards,5,5), dtype="int")
for i in range(0,total_boards):
    boards[i] = np.loadtxt("example.txt", dtype = 'int', skiprows = 2+i*6, max_rows = 5)
print("boards are \n",boards)

boards are 
 [[[22 13 17 11  0]
  [ 8  2 23  4 24]
  [21  9 14 16  7]
  [ 6 10  3 18  5]
  [ 1 12 20 15 19]]

 [[ 3 15  0  2 22]
  [ 9 18 13 17  5]
  [19  8  7 25 23]
  [20 11 10 24  4]
  [14 21 16 12  6]]

 [[14 21 17 24  4]
  [10 16 15  9 19]
  [18  8 23 26 20]
  [22 11 13  6  5]
  [ 2  0 12  3  7]]]


In [138]:
### Step 5
mark = np.zeros((total_boards,5,5),int)
print(mark)

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]


In [48]:
print(calls[4], type(calls[4]))
print(boards[0][0], type(boards[0][0][4]))

for row in range(0,4):
    for i in range(0,4):
        if boards[0][row][i] == calls[4]:
            print(boards[0][0][i])
            print("check")
            mark[0][row][i] = 1
            break
    
print(mark[0][0])
mark[0][0][3]=0
print(mark[0][0])

11 <class 'numpy.int64'>
[22 13 17 11  0] <class 'numpy.int64'>
11
check
[0 0 0 1 0]
[0 0 0 0 0]


In [56]:
print(boards[0][1])

[ 8  2 23  4 24]


In [101]:
print(np.any(boards[0][1], where = [8]))

True


In [57]:
print(sum(boards[0][1]))

61


In [116]:
print(np.sum(boards[0], axis=0))

[58 46 77 64 55]


In [74]:
sum_column_2 = sum(boards[0][i][1] for i in range(0,5))
print(sum_column_2)

46


In [144]:
# according to Marjn's code, it seems that NumPy array can work so smart like this:
print(calls)
print(boards)
mark = np.zeros((total_boards,5,5),int)
check_sum_row = np.zeros((total_boards,1,5),int)
check_sum_column = np.zeros((total_boards,1,5),int)
check_bingo = np.zeros((total_boards,2,5),int)
check_bingo_sum = [0]*total_boards
bingo = 0
for i in range(0,len(calls)):
    print("round ", i+1)
    check_sum_row = np.zeros((total_boards,1,5),int)
    mark[boards == calls[i]] = 1
    print(mark)
    for b in range(0,total_boards):
        check_sum_row[b][0]=[sum(mark[b][row]) for row in range(0,5)]
        print("check_sum_row for", b, " is", check_sum_row[b][0])
        check_sum_column[b][0]=np.sum(mark[b],axis=0)
        print("check_sum_column for", b, " is", check_sum_column[b][0])
        check_bingo[b][0]=np.where(check_sum_row[b][0] == 5, 1, 0)
        check_bingo[b][1]=np.where(check_sum_column[b][0] == 5, 1, 0)
        check_bingo_sum[b]=sum(check_bingo[b][0])+sum(check_bingo[b][1])
        print("check_bingo_sum for", b, " is", check_bingo_sum[b])
        if check_bingo_sum[b] == 1:
            print(mark)
            print("bingo!! winning board is", b+1)
            bingo = 1
            break
    # print("check_sum_row are:\n", check_sum_row)
    # print("check_bingo are:\n", check_bingo)
    # print("check_bingo_sum are:\n", check_bingo_sum)
    if bingo == 1:
        break
# it took long time..
# calculate the sum of the remaining values of the winning board
print(boards[b])
print(boards[b][mark[b]==0])
remaining = boards[b][mark[b] == 0].sum()
print("remaining is ",remaining)
score = remaining * calls[i]
print("score is ", score)

[ 7  4  9  5 11 17 23  2  0 14 21 24 10 16 13  6 15 25 12 22 18 20  8 19
  3 26  1]
[[[22 13 17 11  0]
  [ 8  2 23  4 24]
  [21  9 14 16  7]
  [ 6 10  3 18  5]
  [ 1 12 20 15 19]]

 [[ 3 15  0  2 22]
  [ 9 18 13 17  5]
  [19  8  7 25 23]
  [20 11 10 24  4]
  [14 21 16 12  6]]

 [[14 21 17 24  4]
  [10 16 15  9 19]
  [18  8 23 26 20]
  [22 11 13  6  5]
  [ 2  0 12  3  7]]]
round  1
[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 1]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 1 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 1]]]
check_sum_row for 0  is [0 0 1 0 0]
check_sum_column for 0  is [0 0 0 0 1]
check_bingo_sum for 0  is 0
check_sum_row for 1  is [0 0 1 0 0]
check_sum_column for 1  is [0 0 1 0 0]
check_bingo_sum for 1  is 0
check_sum_row for 2  is [0 0 0 0 1]
check_sum_column for 2  is [0 0 0 0 1]
check_bingo_sum for 2  is 0
round  2
[[[0 0 0 0 0]
  [0 0 0 1 0]
  [0 0 0 0 1]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0

Finally it went!!!!!!!!

Now it is time to try with the real input data.

In [148]:
### Step 1 and 2
# get the "calls" list
calls = np.loadtxt("input.txt", dtype = 'int', delimiter = ',', max_rows = 1)
print("calls are ", calls)
type(calls[0])

### Step 3
# how many boards are there?
boards_pre = np.loadtxt("input.txt", dtype = 'int', skiprows = 2)
total_boards = int(len(boards_pre)/5) 
print("we have ", total_boards, " boards")

### Step 4
# prepare bords to store
boards = np.zeros((total_boards,5,5), dtype="int")
for i in range(0,total_boards):
    boards[i] = np.loadtxt("input.txt", dtype = 'int', skiprows = 2+i*6, max_rows = 5)
# print("boards are \n",boards)

mark = np.zeros((total_boards,5,5),int)
check_sum_row = np.zeros((total_boards,1,5),int)
check_sum_column = np.zeros((total_boards,1,5),int)
check_bingo = np.zeros((total_boards,2,5),int)
check_bingo_sum = [0]*total_boards
bingo = 0
for i in range(0,len(calls)):
    print("round ", i+1)
    check_sum_row = np.zeros((total_boards,1,5),int)
    mark[boards == calls[i]] = 1
    # print(mark)
    for b in range(0,total_boards):
        check_sum_row[b][0]=[sum(mark[b][row]) for row in range(0,5)]
        # print("check_sum_row for", b, " is", check_sum_row[b][0])
        check_sum_column[b][0]=np.sum(mark[b],axis=0)
        # print("check_sum_column for", b, " is", check_sum_column[b][0])
        check_bingo[b][0]=np.where(check_sum_row[b][0] == 5, 1, 0)
        check_bingo[b][1]=np.where(check_sum_column[b][0] == 5, 1, 0)
        check_bingo_sum[b]=sum(check_bingo[b][0])+sum(check_bingo[b][1])
        # print("check_bingo_sum for", b, " is", check_bingo_sum[b])
        if check_bingo_sum[b] == 1:
            print(mark[b])
            print("bingo!! winning board is", b+1)
            bingo = 1
            break
    # print("check_sum_row are:\n", check_sum_row)
    # print("check_bingo are:\n", check_bingo)
    # print("check_bingo_sum are:\n", check_bingo_sum)
    if bingo == 1:
        break
# it took long time..
# calculate the sum of the remaining values of the winning board
print(boards[b])
print(boards[b][mark[b]==0])
remaining = boards[b][mark[b] == 0].sum()
print("remaining is ",remaining)
score = remaining * calls[i]
print("score is ", score)

calls are  [91 17 64 45  8 13 47 19 52 68 63 76 82 44 28 56 37  2 78 48 32 58 72 53
  9 85 77 89 36 22 49 86 51 99  6 92 80 87  7 25 31 66 84  4 98 67 46 61
 59 79  0  3 38 27 23 95 20 35 14 30 26 33 42 93 12 57 11 54 50 75 90 41
 88 96 40 81 24 94 18 39 70 34 21 55  5 29 71 83  1 60 74 69 10 62 43 73
 97 65 15 16]
we have  100  boards
round  1
round  2
round  3
round  4
round  5
round  6
round  7
round  8
round  9
round  10
round  11
round  12
round  13
round  14
round  15
round  16
round  17
round  18
round  19
round  20
round  21
round  22
round  23
round  24
round  25
round  26
round  27
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 1 1 1 1]]
bingo!! winning board is 16
[[82  5 38 41 95]
 [70 52 11 21 51]
 [81 20  0 14 83]
 [57 36 60 59 42]
 [77 13 85 32 63]]
[ 5 38 41 95 70 11 21 51 81 20  0 14 83 57 36 60 59 42]
remaining is  784
score is  60368


## Problem 2

Find the last winning board and get the score of the board.

In [154]:
bool_test = np.zeros((2,3),bool)
print(bool_test)
bool_test[1][2]=True
for i in range(3):
    if bool_test[1][i] != bool_test[0][i]:
        print(i," is the last")

[[False False False]
 [False False False]]
2  is the last


In [156]:
### Step 1 and 2
# get the "calls" list
calls = np.loadtxt("example.txt", dtype = 'int', delimiter = ',', max_rows = 1)
print("calls are ", calls)
type(calls[0])

### Step 3
# how many boards are there?
boards_pre = np.loadtxt("example.txt", dtype = 'int', skiprows = 2)
total_boards = int(len(boards_pre)/5) 
print("we have ", total_boards, " boards")

### Step 4
# prepare bords to store
boards = np.zeros((total_boards,5,5), dtype="int")
for i in range(0,total_boards):
    boards[i] = np.loadtxt("example.txt", dtype = 'int', skiprows = 2+i*6, max_rows = 5)
# print("boards are \n",boards)

mark = np.zeros((total_boards,5,5),int)
check_sum_row = np.zeros((total_boards,1,5),int)
check_sum_column = np.zeros((total_boards,1,5),int)
check_bingo = np.zeros((total_boards,2,5),int)
check_bingo_sum = [0]*total_boards
bingo = 0
remaining = np.zeros((len(calls),total_boards),bool)
# we need the
for i in range(0,len(calls)):
    print("round ", i+1)
    check_sum_row = np.zeros((total_boards,1,5),int)
    mark[boards == calls[i]] = 1
    # print(mark)
    for b in range(0,total_boards):
        check_sum_row[b][0]=[sum(mark[b][row]) for row in range(0,5)]
        # print("check_sum_row for", b, " is", check_sum_row[b][0])
        check_sum_column[b][0]=np.sum(mark[b],axis=0)
        # print("check_sum_column for", b, " is", check_sum_column[b][0])
        check_bingo[b][0]=np.where(check_sum_row[b][0] == 5, 1, 0)
        check_bingo[b][1]=np.where(check_sum_column[b][0] == 5, 1, 0)
        check_bingo_sum[b]=sum(check_bingo[b][0])+sum(check_bingo[b][1])
        # print("check_bingo_sum for", b, " is", check_bingo_sum[b])
        if check_bingo_sum[b] >= 1:
            # print(mark[b])
            remaining[i][b] = True
            # print("bingo!! winning board is", b+1)
            bingo = bingo + 1            
    # print("check_sum_row are:\n", check_sum_row)
    # print("check_bingo are:\n", check_bingo)
    # print("check_bingo_sum are:\n", check_bingo_sum)
    if bingo == total_boards:
        for b in range(0,total_boards):
            if remaining[i][b] != remaining[i-1][b]:
                print("the last board that won is ", b+1)
                break
        break
    else: 
        bingo = 0
# it took long time..
# calculate the sum of the remaining values of the winning board
print(boards[b])
print(boards[b][mark[b]==0])
remaining_nr_sum = boards[b][mark[b] == 0].sum()
print("the sum of the remaining nr is ",remaining_nr_sum)
score = remaining_nr_sum * calls[i]
print("score is ", score)

calls are  [ 7  4  9  5 11 17 23  2  0 14 21 24 10 16 13  6 15 25 12 22 18 20  8 19
  3 26  1]
we have  3  boards
round  1
round  2
round  3
round  4
round  5
round  6
round  7
round  8
round  9
round  10
round  11
round  12
round  13
round  14
round  15
the last board that won is  2
[[ 3 15  0  2 22]
 [ 9 18 13 17  5]
 [19  8  7 25 23]
 [20 11 10 24  4]
 [14 21 16 12  6]]
[ 3 15 22 18 19  8 25 20 12  6]
the sum of the remaining nr is  148
score is  1924


OK, it seems that it went fine.
Let's try with the real input.

In [157]:
### Step 1 and 2
# get the "calls" list
calls = np.loadtxt("input.txt", dtype = 'int', delimiter = ',', max_rows = 1)
print("calls are ", calls)
type(calls[0])

### Step 3
# how many boards are there?
boards_pre = np.loadtxt("input.txt", dtype = 'int', skiprows = 2)
total_boards = int(len(boards_pre)/5) 
print("we have ", total_boards, " boards")

### Step 4
# prepare bords to store
boards = np.zeros((total_boards,5,5), dtype="int")
for i in range(0,total_boards):
    boards[i] = np.loadtxt("input.txt", dtype = 'int', skiprows = 2+i*6, max_rows = 5)
# print("boards are \n",boards)

mark = np.zeros((total_boards,5,5),int)
check_sum_row = np.zeros((total_boards,1,5),int)
check_sum_column = np.zeros((total_boards,1,5),int)
check_bingo = np.zeros((total_boards,2,5),int)
check_bingo_sum = [0]*total_boards
bingo = 0
remaining = np.zeros((len(calls),total_boards),bool)
# we need the
for i in range(0,len(calls)):
    print("round ", i+1)
    check_sum_row = np.zeros((total_boards,1,5),int)
    mark[boards == calls[i]] = 1
    # print(mark)
    for b in range(0,total_boards):
        check_sum_row[b][0]=[sum(mark[b][row]) for row in range(0,5)]
        # print("check_sum_row for", b, " is", check_sum_row[b][0])
        check_sum_column[b][0]=np.sum(mark[b],axis=0)
        # print("check_sum_column for", b, " is", check_sum_column[b][0])
        check_bingo[b][0]=np.where(check_sum_row[b][0] == 5, 1, 0)
        check_bingo[b][1]=np.where(check_sum_column[b][0] == 5, 1, 0)
        check_bingo_sum[b]=sum(check_bingo[b][0])+sum(check_bingo[b][1])
        # print("check_bingo_sum for", b, " is", check_bingo_sum[b])
        if check_bingo_sum[b] >= 1:
            # print(mark[b])
            remaining[i][b] = True
            # print("bingo!! winning board is", b+1)
            bingo = bingo + 1            
    # print("check_sum_row are:\n", check_sum_row)
    # print("check_bingo are:\n", check_bingo)
    # print("check_bingo_sum are:\n", check_bingo_sum)
    if bingo == total_boards:
        for b in range(0,total_boards):
            if remaining[i][b] != remaining[i-1][b]:
                print("the last board that won is ", b+1)
                break
        break
    else: 
        bingo = 0
# it took long time..
# calculate the sum of the remaining values of the winning board
print(boards[b])
print(boards[b][mark[b]==0])
remaining_nr_sum = boards[b][mark[b] == 0].sum()
print("the sum of the remaining nr is ",remaining_nr_sum)
score = remaining_nr_sum * calls[i]
print("score is ", score)

calls are  [91 17 64 45  8 13 47 19 52 68 63 76 82 44 28 56 37  2 78 48 32 58 72 53
  9 85 77 89 36 22 49 86 51 99  6 92 80 87  7 25 31 66 84  4 98 67 46 61
 59 79  0  3 38 27 23 95 20 35 14 30 26 33 42 93 12 57 11 54 50 75 90 41
 88 96 40 81 24 94 18 39 70 34 21 55  5 29 71 83  1 60 74 69 10 62 43 73
 97 65 15 16]
we have  100  boards
round  1
round  2
round  3
round  4
round  5
round  6
round  7
round  8
round  9
round  10
round  11
round  12
round  13
round  14
round  15
round  16
round  17
round  18
round  19
round  20
round  21
round  22
round  23
round  24
round  25
round  26
round  27
round  28
round  29
round  30
round  31
round  32
round  33
round  34
round  35
round  36
round  37
round  38
round  39
round  40
round  41
round  42
round  43
round  44
round  45
round  46
round  47
round  48
round  49
round  50
round  51
round  52
round  53
round  54
round  55
round  56
round  57
round  58
round  59
round  60
round  61
round  62
round  63
round  64
round  65
round  66
round  67
r

Yaay, got the right answer!!!